# Building and analysing a paper co-authorship network

In [2]:
import graph_tool_extras as gte
from pathlib import Path

## Introduction

In this notebook, a network of co-authorship of papers about condensed matter authors was built. The database in the file below is used. It's from the Stanford Network Analysis Project's dataset at https://networks.skewed.de/net/arxiv_authors#CondMat_draw.

In [3]:
FOLDER_PATH = Path.cwd() / 'archive'
EDGES_FILE_PATH = FOLDER_PATH / 'musae_git_edges.csv'

## Understanding the data

| Concept of vertices                                                                                                 | Concept of edges                                                                                                                             | Operationalization of vertices                                                                                                                                                                                                                                                           | Operationalization of edges                                                                                                                                                                                                                                                  |
|---------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Webpage                                                                                                             | An edge between i and j indicates co-authorship between i and j.                                                                           | Each vertex represents an author of a paper in the cond-mat category on the arXiv platform. The data was collected from January 1993 to April 2003.                                                                                                                                 | An edge between authors i and j indicates a co-authorship relationship between them. In cases where other k authors also contributed to the production of the paper, these additional k authors are connected to both authors i and j, as well as to each other, through edges. |


## Creating functions to build the network

In [4]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
    return u

In [5]:
def get_or_add_edge(g, author_a, author_b):
    e = g.edge_by_ids(author_a, author_b)
    if e is None:
        e = g.add_edge_by_ids(author_a, author_b)
    return e

## Reading the data and building the network

In [7]:
g = gte.Graph(directed=False)

In [10]:
with open(EDGES_FILE_PATH) as file:

    next(file)
    
    for line in file:
        parts = line.split(',')

        author_a = parts[0]
        author_b = parts[1].replace('\n', '')
        
        vertex_a = get_or_add_vertex(g, author_a)
        vertex_b = get_or_add_vertex(g, author_b)
        edge = get_or_add_edge(g, author_a, author_b)

In [11]:
g = gte.clean(g)

In [12]:
gte.save(g, 'github_social.net.gz')

## Configuring the layout and rendering the network

In [13]:
from graph_tool import draw
import netpixi

In [14]:
layout = draw.sfdp_layout(g)

In [15]:
gte.move(g, layout)

In [16]:
gte.save(g, 'github_social_layout.net.gz')

In [17]:
r = netpixi.render('github_social_layout.net.gz', infinite=True)

## Improving network vizualization

In [24]:
r.vertex_default(
    size=2,
    color=0xff0000,
    bwidth=1,
    bcolor=0x007700,
)

In [20]:
r.edge_default(
    width=0.2,
    color=0xffffff,
    curve1=0,
    curve2=0,
)

## Analyzing the network